<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/*water_rights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Water Rights in the Treasure Valley for Diversions

By: Carli Beisel


Created on August 2024


Purpose:

1) Crop Water Rights layer to drainshds in the Treasure Valley and also create a separate file that organizes all WR based on priorty year + surface/groundwater.


2) Create a figure showing the difference between surface + groundwater rights in diversions.


## 1. Organize IDWR Water Right Data from GIS Data Hub

Separate based on surface/groundwater rights and organize based on priority year.  

In [ ]:
# -------------------- #
#   Import Packages    #
# -------------------- #
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import glob

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ----------------------------------- #
#     Rename Drains in LBRB           #
# ----------------------------------- #

# Function to rename names in the 'Name' column of a shapefile
def rename_names_in_shapefile(shapefile_path, output_path, name_mapping):
    """
    Rename names in the 'Name' column of a shapefile.

    Parameters:
    shapefile_path (str): Path to the input shapefile.
    output_path (str): Path to save the modified shapefile.
    name_mapping (dict): Dictionary where keys are old names and values are new names.

    """
    gdf = gpd.read_file(shapefile_path)
    if 'Name' not in gdf.columns:
        raise ValueError("The shapefile does not contain a 'Name' column.")
    gdf['Name'] = gdf['Name'].replace(name_mapping)
    gdf.to_file(output_path)
    print(f"Shapefile with updated names saved to {output_path}")

if __name__ == "__main__":
    input_shapefile = '/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_Merge_07072022.shp'
    output_shapefile = '/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_08162024.shp'

    # Dictionary with old names as keys and new names as values
    name_changes = {
        'Dixie drain': 'Dixie Drain',
        'East Hartley Drain': 'East Hartley Gulch',
        'West Hartley': 'West Hartley Gulch',
        'Drainage District No. 3': 'Drainage District No3'
    }

    rename_names_in_shapefile(input_shapefile, output_shapefile, name_changes)

In [ ]:
# ----------------------------------- #
#         Crop to the LBRB            #
# ----------------------------------- #
water_rights = gpd.read_file('/content/drive/MyDrive/Data/Model Modifications/water_rights/WaterRightPOUs/WaterRightPOUs.shp')
lbrb = gpd.read_file('/content/drive/MyDrive/Data/GIS Shapefiles/LBRB_shp/LBRB_file.shp')

lbrb_water_rights = gpd.overlay(water_rights, lbrb, how='intersection')

lbrb_water_rights.to_file('/content/drive/MyDrive/Data/Model Modifications/water_rights/wr_lbrb_masked/lbrb_water_rights.shp')

In [ ]:
# ----------------------------------- #
#       Convert .SHP to .CSV          #
# ----------------------------------- #
lbrb_water_rights = gpd.read_file('/content/drive/MyDrive/Data/Model Modifications/water_rights/lbrb_water_rights/lbrb_water_rights.shp')

wr = pd.DataFrame(water_rights.drop(columns='geometry'))
wr.to_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/lbrb_water_rights.csv', index=False)

In [ ]:
# ----------------------------------- #
#       Crop WR to Drainsheds         #
# ----------------------------------- #
water_rights = gpd.read_file('/content/drive/MyDrive/Data/Model Modifications/water_rights/WaterRightPOUs/WaterRightPOUs.shp')
drainsheds = gpd.read_file('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_08162024.shp')
names = drainsheds['Name']
water_rights = water_rights.to_crs(drainsheds.crs)
output_dir = '/content/drive/MyDrive/Data/Model Modifications/water_rights/wr_drainshed_masked/'

# Create function to crop water rights to each drainshed area
def crop_water_rights_to_drainsheds(water_rights, drainsheds, output_dir):
    mask_gdf = drainsheds
    for i, mask_feature in mask_gdf.iterrows():
        mask_geom = mask_feature['geometry']
        mask_name = mask_feature['Name']

        cropped_gdf = gpd.overlay(water_rights, gpd.GeoDataFrame(geometry=[mask_geom], crs=mask_gdf.crs), how='intersection')

        output_filename = os.path.join(output_dir, f"{mask_name}_wr.shp")

        # Save the cropped shapefile
        cropped_gdf.to_file(output_filename)
        print(f"Cropped shapefile saved to {output_filename}")

crop_water_rights_to_drainsheds(water_rights, drainsheds, output_dir)


In [ ]:
# ----------------------------------- #
#   Convert Drain WR to CSV File      #
# ----------------------------------- #

shapefiles = glob.glob('/content/drive/MyDrive/Data/Model Modifications/water_rights/wr_drainshed_masked/*.shp')

csv_output_dir = '/content/drive/MyDrive/Data/Model Modifications/water_rights/wr_drainshed_masked/'
for shapefile in shapefiles:
    gdf = gpd.read_file(shapefile)
    base_name = os.path.basename(shapefile)
    drainshed_name = base_name.split('_wr')[0]  # Extracting the part before '_wr'
    gdf['Drainshed'] = drainshed_name #add a column for each drainshed name
    csv_filename = os.path.splitext(base_name)[0] + '.csv'
    csv_filepath = os.path.join(csv_output_dir, csv_filename)
    gdf.to_csv(csv_filepath, index=False)
    print(f"Converted {shapefile} to {csv_filepath} with added 'Drainshed' column.")

In [ ]:
# ----------------------------------------- #
#      Merge drainhed WRs into one file     #
# ----------------------------------------- #

csv_files = glob.glob('/content/drive/MyDrive/Data/Model Modifications/water_rights/wr_drainshed_masked/*.csv')
df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
merged_df = pd.concat(df_list, ignore_index=True)
merged_df.to_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv', index=False)

#add in new column to separate GW from SW
file = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv')
file['Type'] = file['Source'].apply(lambda x: 'GW' if x == 'GROUND WATER' else 'SW')
file.to_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv', index=False)

In [ ]:
# ------------------------------ #
#      Drain WRs Model Input     #
# ------------------------------ #
# based on SW + GW + total

file_path = '/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv'
df = pd.read_csv(file_path)

# Convert 'PriorityDa' column to datetime format and extract the year
df['PriorityDa'] = pd.to_datetime(df['PriorityDa'], format='%Y-%m-%d', errors='coerce')
df['Year'] = df['PriorityDa'].dt.year

# Group by year, type, and drainage area (assuming 'Drainshed' is the column for drainage)
yearly_priority_counts = df.groupby(['Year', 'Type', 'Drainshed']).agg({'PriorityDa': 'nunique'}).reset_index()
yearly_priority_counts.rename(columns={'PriorityDa': 'Water Rights'}, inplace=True)

# Pivot the table to create separate columns for GW and SW water rights for each drainage
pivot_counts = yearly_priority_counts.pivot_table(index=['Year', 'Drainshed'], columns='Type', values='Water Rights', fill_value=0).reset_index()

pivot_counts.columns = ['Year', 'Drainshed', 'GW Water Rights', 'SW Water Rights']

# Add a column for total water rights (GW + SW)
pivot_counts['Total Water Rights'] = pivot_counts['GW Water Rights'] + pivot_counts['SW Water Rights']

# Calculate cumulative water rights (GW, SW, Total) for each drainage area individually
pivot_counts['gw_wr'] = pivot_counts.groupby('Drainshed')['GW Water Rights'].cumsum()
pivot_counts['sw_wr'] = pivot_counts.groupby('Drainshed')['SW Water Rights'].cumsum()
pivot_counts['total_wr'] = pivot_counts.groupby('Drainshed')['Total Water Rights'].cumsum()

# Create a new DataFrame that stores this information
wr_final = pivot_counts

columns_to_save = [
    'Year',         # Year
    'Drainshed',               # Drainage area
    'gw_wr',    # Cumulative Groundwater Water Rights
    'sw_wr',    # Cumulative Surface Water Rights
    'total_wr'  # Cumulative Total Water Rights
]

# Save the selected columns to a new CSV file
output_file_path = '/content/drive/MyDrive/Data/Model Modifications/water_rights/final_wr.csv'
wr_final[columns_to_save].to_csv(output_file_path, index=False)

## Figures

In [ ]:
# ----------------------------------- #
#   SW/GW Comparison in Drainsheds
#       Decreed Year
# ----------------------------------- #

#violin plot of all drainages

file = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv')

# Convert DecreedDat to datetime format and get year
file['DecreedDat'] = pd.to_datetime(file['DecreedDat'])
file['DecreedYear'] = file['DecreedDat'].dt.year

plot = sns.catplot(
    data=file, x="Drainshed", y="DecreedYear", hue="Type",
    kind="violin", height=8, aspect=2)  # Increase height and aspect ratio for larger plot

plot.set_xticklabels(rotation=45, ha='right')
plot.fig.suptitle("Distribution of Water Rights for Drainshed by Decreed Year", fontsize=16)
plot.fig.subplots_adjust(top=0.9)

# Show the plot
plt.show()

In [ ]:
# ----------------------------------- #
#   SW/GW Comparison in Drainsheds
#       Priority Year
# ----------------------------------- #

#violin plot of all drainages

# Load the data
file = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/drainsheds_wr.csv')

# Convert DecreedDat to datetime format
file['PriorityDa'] = pd.to_datetime(file['PriorityDa'])

# Extract the year from DecreedDat
file['PriorityYear'] = file['PriorityDa'].dt.year

# Plot using sns.catplot
plot = sns.catplot(
    data=file, x="Drainshed", y="PriorityYear", hue="Type",
    kind="violin", height=8, aspect=2)  # Increase height and aspect ratio for larger plot

plot.set_xticklabels(rotation=45, ha='right')
plot.fig.suptitle("Distribution of Water Rights for Drainshed by Priority Year", fontsize=16)
plot.fig.subplots_adjust(top=0.9)

# Show the plot
plt.show()